In [1]:
!pip install pandas


In [34]:
import pandas as pd
import numpy as np
import re

In [35]:
%load_ext gams.magic
m = gams.exchange_container

The gams.magic extension is already loaded. To reload it, use:
  %reload_ext gams.magic


In [37]:
gams.reset()
m = gams.exchange_container
df = pd.read_csv('FInalFoodData.csv', index_col="Item")
df 

,Description,Brand,Calories,Serving Size,Nutrients,Nov 2023 Price in Dollars,Percent Change from Nov 2022,Percent Change from Oct 2023
Item,,,,,,,,
chicken,CHICKEN,Keystone Brand Meats Inc,107.0 kcal,56.0 g,"Fatty acids, total trans: 0.0 g, Carbohydrate,...",0.12/oz,3.1,2.6
yogurt,YOGURT,Agri-Service LLC,94.0 kcal,170.0 ml,"Iron, Fe: 0.0 mg, Calcium, Ca: 118.0 mg, Fiber...",0.07/oz,NaN,NaN
cereal,CEREAL,"General Mills, Inc.",387.0 kcal,31.0 g,"Fatty acids, total polyunsaturated: 1.61 g, Ir...",0.27/oz,NaN,NaN
Corn,CORN,"Kcb Industries, L.L.C.",545.0 kcal,33.0 g,"Vitamin D (D2 + D3), International Units: 0.0 ...",0.17/oz,NaN,NaN
Potatoes,POTATOES,THE LITTLE POTATO COMPANY,74.0 kcal,148.0 g,"Iron, Fe: 0.73 mg, Cholesterol: 0.0 mg, Calciu...",0.05/oz,2.7,-2.8
...,...,...,...,...,...,...,...,...
Ice cream,ICE CREAM,"Clover Stornetta Farms, Inc.",205.0 kcal,88.0 g,"Energy: 205.0 kcal, Carbohydrate, by differenc...",0.17/oz,NaN,NaN
cookies,COOKIES,KNO,441.0 kcal,100.0 g,"Fiber, total dietary: 0.0 g, Sodium, Na: 0.0 m...",0.20/oz,NaN,NaN
brownies,BROWNIES,TWO-BITE,460.0 kcal,50.0 g,"Iron, Fe: 2.88 mg, Fiber, total dietary: 0.0 g...",0.34/oz,NaN,NaN


In [4]:
df['Calories'] = df['Calories'].str.extract('(\d+\.\d+)').astype(float)

In [5]:
df['Serving Size'] = df['Serving Size'].str.extract('(\d+\.\d+)').astype(float)

In [6]:
def extract_nutrients(row):
    nutrients = {}
    items = re.split(', ', row['Nutrients'])
    for item in items:
        match = re.match(r'(.+?): (.+?) (.+)', item)
        if match:
            nutrient, value = match.groups()[:2]
            nutrients[nutrient] = float(value) if '.' in value else int(value)
    return pd.Series(nutrients)

# Apply the function to each row and concatenate the result with the original DataFrame
df = pd.concat([df, df.apply(extract_nutrients, axis=1)], axis=1)

# Drop the original 'Nutrients' column
df = df.drop('Nutrients', axis=1)

In [7]:
df.head()

,Description,Brand,Calories,Serving Size,Alanine,Arginine,Ash,Aspartic acid,Ca,Caffeine,...,other,soluble,total,total ascorbic acid,total dietary,total including NLEA,total monounsaturated,total polyunsaturated,total saturated,total trans
Item,,,,,,,,,,,,,,,,,,,,,
chicken,CHICKEN,Keystone Brand Meats Inc,107.0,56.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,NaN,0.89,0.0
yogurt,YOGURT,Agri-Service LLC,94.0,170.0,NaN,NaN,NaN,NaN,118.0,NaN,...,NaN,NaN,NaN,0.7,0.0,15.88,NaN,NaN,1.76,NaN
cereal,CEREAL,"General Mills, Inc.",387.0,31.0,NaN,NaN,NaN,NaN,323.0,NaN,...,48.0,NaN,NaN,19.4,6.5,29.03,1.61,1.61,0.00,0.0
Corn,CORN,"Kcb Industries, L.L.C.",545.0,33.0,NaN,NaN,NaN,NaN,61.0,NaN,...,NaN,NaN,NaN,NaN,6.1,24.24,NaN,NaN,4.55,0.0
Potatoes,POTATOES,THE LITTLE POTATO COMPANY,74.0,148.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,2.4,2.0,0.68,NaN,NaN,0.00,0.0


In [8]:
df.rename(columns={'by difference': 'Carbs'}, inplace=True)
df.rename(columns={'Total lipid (fat)': 'Fat'}, inplace=True)
df.rename(columns={'Ca': 'Calcium'}, inplace=True)
df.rename(columns={'Vitamin D (D2 + D3)': 'Vitamin D'}, inplace=True)
df.rename(columns={'Serving Size': 'Serving'}, inplace=True)

In [9]:
df = df[["Calories", "Serving", 'Protein', "Carbs", "Fat", "Calcium"]]


In [38]:
# Load the actual prices from the CSV file
food_prices_df = pd.read_csv('FInalFoodData.csv')
food_prices_df['Price per Ounce'] = food_prices_df['Nov 2023 Price in Dollars'].str.extract(r'(\d+\.\d+)/oz').astype(float)

# Create a dictionary to map food items to their prices
food_prices = dict(zip(food_prices_df['Item'], food_prices_df['Nov 2023 Price in Dollars']))

# Update the dataframe with the actual prices
df['Price'] = df.index.map(food_prices)


In [39]:
df_limited = df.head(10)

In [40]:
df.fillna(0, inplace=True)

In [41]:
df = df.drop(['Rice Krispies', 'chili', 'Cornbread', "Instant oatmeal", 'Cheese curds', 'Peanut butter and jelly sandwiches'])
# df = df.drop(['chili'])
# df = df.drop(['Cornbread', "Instant oatmeal"])
# df = df.drop(['Peanut butter and jelly sandwiches'])

In [42]:
foods = df.index.values.tolist()
foods

['chicken',
 'yogurt',
 'cereal',
 'Corn',
 'Potatoes',
 'Green beans',
 'Carrots',
 'Tomatoes',
 'Bell peppers',
 'Onions',
 'Lettuce',
 'Spinach',
 'Broccoli',
 'Cauliflower',
 'Cucumbers',
 'Zucchini',
 'Peas',
 'Brussels sprouts',
 'Apples',
 'Bananas',
 'Grapes',
 'Oranges',
 'Strawberries',
 'Blueberries',
 'Watermelon',
 'Pears',
 'Cherries',
 'Plums',
 'Cantaloupe',
 'Raspberries',
 'Blackberries',
 'Kiwi',
 'Popcorn',
 'Pretzels',
 'Trail mix',
 'Beef jerky',
 'Potato chips',
 'Granola bars',
 'Applesauce',
 'peanuts',
 'almonds',
 'Crackers',
 'Cheese slices',
 'Fruit cups',
 'marshmallows',
 'chocolate',
 'graham crackers',
 'Milk',
 'Cheese',
 'steaks',
 'burgers',
 'Pork (bacon, ham)',
 'Eggs',
 'Wheat bread',
 'Oatmeal',
 'Pancake mix',
 'Pasta',
 'Rice',
 'Peaches',
 'Instant noodles',
 'Frozen pizza',
 'mac and cheese',
 'Canned soups',
 'cream cheese',
 'Chips and salsa',
 'Fruit snacks',
 'dried fruits',
 'String cheese',
 'Protein bar',
 'Coffee',
 'tea',
 'Energy dr

In [43]:
meats = ['chicken', 'steaks', 'burgers', 'Pork (bacon, ham)']
fruits = ['Apples','Bananas','Grapes','Oranges','Strawberries','Blueberries','Watermelon','Pears','Cherries','Plums','Cantaloupe','Raspberries','Blackberries','Kiwi','Peaches']  
vegetables = ['Corn','Potatoes','Green beans','Carrots','Tomatoes','Bell peppers','Onions','Lettuce','Spinach','Broccoli','Cauliflower','Cucumbers','Zucchini','Peas','Brussels sprouts','Salads']        
breakfast = ['yogurt','cereal', 'Eggs', 'Oatmeal','Pancake mix', 'cream cheese','Toaster pastries','Frozen waffles']  
frozen = ['Frozen pizza', 'Pizza rolls','mozzarella sticks','frozen vegetables']
drinks = ['Coffee','tea','Energy drinks','sports drinks','Fruit juices','smoothies','Soda']
desserts = ['Ice cream','cookies','brownies', 'marshmallows','chocolate'] 
snacks = ['Popcorn','Pretzels','Trail mix','Beef jerky','Potato chips','Granola bars','Applesauce','peanuts','almonds','Crackers','Cheese slices','Fruit cups','graham crackers', "Hummus",'Chips and salsa','Fruit snacks','dried fruits',"String cheese",'Protein bar']
dry = ["pita bread", 'Instant noodles', "mac and cheese", 'Pasta','Rice', 'Wheat bread','Canned soups']   
dairy = ['Cheese slices', 'Milk','Cheese',]
    

In [44]:
len(meats) + len(fruits) + len(vegetables) + len(breakfast) + len(frozen) + len(drinks) + len(desserts) + len(snacks) + len(dry)

85

In [45]:
len(df)


87

In [46]:
leftovers = []
for i in foods:
    if i in meats:
        continue
    if i in fruits:
        continue
    if i in vegetables:
        continue
    if i in breakfast:
        continue
    if i in frozen:
        continue
    if i in drinks:
        continue
    if i in desserts:
        continue
    if i in snacks:
        continue
    if i in dry:
        continue
    if i in dairy:
        continue
    
leftovers

[]

In [47]:
import pandas as pd

# Assuming df is your original DataFrame
# Assuming meats, fruits, vegetables, etc. are lists containing food items of each type

# Create an empty dictionary to store DataFrames for each type of food
food_dict = {}

# Iterate through the food items in the original DataFrame
for i in df.index.values:
    if i in meats:
        if 'Meats' not in food_dict:
            food_dict['Meats'] = pd.DataFrame(columns=df.columns)
        food_dict['Meats'] = pd.concat([food_dict['Meats'], df.loc[[i]]])
    elif i in fruits:
        if 'Fruits' not in food_dict:
            food_dict['Fruits'] = pd.DataFrame(columns=df.columns)
        food_dict['Fruits'] = pd.concat([food_dict['Fruits'], df.loc[[i]]])
    elif i in vegetables:
        if 'Vegetables' not in food_dict:
            food_dict['Vegetables'] = pd.DataFrame(columns=df.columns)
        food_dict['Vegetables'] = pd.concat([food_dict['Vegetables'], df.loc[[i]]])
    elif i in breakfast:
        if 'Breakfast' not in food_dict:
            food_dict['Breakfast'] = pd.DataFrame(columns=df.columns)
        food_dict['Breakfast'] = pd.concat([food_dict['Breakfast'], df.loc[[i]]])
    elif i in frozen:
        if 'Frozen' not in food_dict:
            food_dict['Frozen'] = pd.DataFrame(columns=df.columns)
        food_dict['Frozen'] = pd.concat([food_dict['Frozen'], df.loc[[i]]])
    elif i in drinks:
        if 'Drinks' not in food_dict:
            food_dict['Drinks'] = pd.DataFrame(columns=df.columns)
        food_dict['Drinks'] = pd.concat([food_dict['Drinks'], df.loc[[i]]])
    elif i in desserts:
        if 'Desserts' not in food_dict:
            food_dict['Desserts'] = pd.DataFrame(columns=df.columns)
        food_dict['Desserts'] = pd.concat([food_dict['Desserts'], df.loc[[i]]])
    elif i in snacks:
        if 'Snacks' not in food_dict:
            food_dict['Snacks'] = pd.DataFrame(columns=df.columns)
        food_dict['Snacks'] = pd.concat([food_dict['Snacks'], df.loc[[i]]])
    elif i in dry:
        if 'Dry' not in food_dict:
            food_dict['Dry'] = pd.DataFrame(columns=df.columns)
        food_dict['Dry'] = pd.concat([food_dict['Dry'], df.loc[[i]]])
    elif i in dairy:
        if 'Dairy' not in food_dict:
            food_dict['Dairy'] = pd.DataFrame(columns=df.columns)
        food_dict['Dairy'] = pd.concat([food_dict['Dairy'], df.loc[[i]]])

# Now, the food_dict dictionary contains DataFrames for each type of food
# Access them like food_dict['Meats'], food_dict['Fruits'], etc.


In [48]:
food_dict

{'Meats':                                  Description                     Brand  \
 chicken                              CHICKEN  Keystone Brand Meats Inc   
 steaks                          Pepper steak             Not specified   
 burgers                              BURGERS           Origin Pork LLC   
 Pork (bacon, ham)  Ham or pork with stuffing             Not specified   
 
                      Calories Serving Size  \
 chicken            107.0 kcal       56.0 g   
 steaks               145 kcal    None None   
 burgers            252.0 kcal      151.0 g   
 Pork (bacon, ham)    143 kcal    None None   
 
                                                            Nutrients  \
 chicken            Fatty acids, total trans: 0.0 g, Carbohydrate,...   
 steaks             Protein: 11.9 g, Total lipid (fat): 9.12 g, Ca...   
 burgers            Iron, Fe: 1.92 mg, Fatty acids, total saturate...   
 Pork (bacon, ham)  Protein: 6.26 g, Total lipid (fat): 8.27 g, Ca...   
 
          

In [49]:
meats_df = food_dict["Meats"]
fruits_df = food_dict["Fruits"] 
vegetables_df = food_dict["Vegetables"]
breakfast_df = food_dict['Breakfast'] 
frozen_df = food_dict["Frozen"]
drinks_df = food_dict["Drinks"]
desserts_df = food_dict["Desserts"]
snacks_df = food_dict["Snacks"]
dry_df = food_dict["Dry"]   
dairy_df = food_dict["Dairy"]

In [50]:
dairy_df

,Description,Brand,Calories,Serving Size,Nutrients,Nov 2023 Price in Dollars,Percent Change from Nov 2022,Percent Change from Oct 2023,Price
Milk,MILK,Elmhurst Milk & Cream Co.Inc.,55.0 kcal,236.0 ml,"Fiber, total dietary: 0.0 g, Vitamin C, total ...",0.02/oz,0.0,0.0,0.02/oz
Cheese,CHEESE,Savencia Cheese USA LLC,367.0 kcal,30.0 GRM,"Potassium, K: 97.0 mg, Iron, Fe: 0.0 mg, Fiber...",0.27/oz,-3.3,0.3,0.27/oz


In [51]:
# Item = m.addSet('Item',records=df_limited.index)
# p = m.addParameter('price',[Item],records=df_limited[["Price"]].to_numpy())
# cal = m.addParameter('calories',[Item],records=df_limited[["Calories"]].to_numpy())
# pro = m.addParameter('protein',[Item],records=df_limited[["Protein"]].to_numpy())
# carbs = m.addParameter('carbs',[Item],records=df_limited[["Carbs"]].to_numpy())
# fat = m.addParameter('fat',[Item],records=df_limited[["Fat"]].to_numpy())
# calcium = m.addParameter('calcium',[Item],records=df_limited[["Calcium"]].to_numpy())
# display(calcium.records)

In [57]:
# Assuming df is your existing DataFrame with the "Nov 2023 Price in Dollars" column
# Extract the numeric part of the price

# Reset index if the item names are the index
df = df.reset_index()

# Assuming the column name for item names is now 'index'
new_df = df[['Item', 'Price']]

new_df['Price'] = df['Nov 2023 Price in Dollars'].str.extract(r'(\d+\.\d+)').astype(float)

# For the GAMS parameter, convert the DataFrame to numpy array
price_parameter_data = new_df.to_numpy()

# Assuming m is your GAMS model instance
price = m.addParameter('price', 'Item', records=price_parameter_data)


/tmp/ipykernel_850755/1342029573.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Price'] = df['Nov 2023 Price in Dollars'].str.extract(r'(\d+\.\d+)').astype(float)


ValueError: Attempting to add a symbol named `price` but one already exists in the Container. Symbol replacement is only possible if the symbol is first removed from the Container with the removeSymbols() method. Overwriting symbol 'records' and 'description' are possible if all other properties have not changed.

In [52]:
Item = m.addSet('Item',records=df.index)
me = m.addSet("meat",domain = [Item],records=meats_df.index)
fruit = m.addSet("fruit",domain = [Item],records=fruits_df.index)
veg = m.addSet("vegetable",domain = [Item],records=vegetables_df.index)
bf = m.addSet("breakfast",domain = [Item],records=breakfast_df.index)
fr = m.addSet("frozen",domain = [Item],records=frozen_df.index)
dr = m.addSet("drink",domain = [Item],records=drinks_df.index)
de = m.addSet("dessert",domain = [Item],records=desserts_df.index)
dry = m.addSet("dry",domain = [Item],records=dry_df.index)
dairy = m.addSet("dairy",domain = [Item],records=dairy_df.index)
p = m.addParameter('price',[Item],records=df[["Price"]].to_numpy())
cal = m.addParameter('calories',[Item],records=df[["Calories"]].to_numpy())
pro = m.addParameter('protein',[Item],records=df[["Protein"]].to_numpy())
carbs = m.addParameter('carbs',[Item],records=df[["Carbs"]].to_numpy())
fat = m.addParameter('fat',[Item],records=df[["Fat"]].to_numpy())
calcium = m.addParameter('calcium',[Item],records=df[["Calcium"]].to_numpy())
price = m.addParameter('price',[Item],records=df[["Nov 2023 Price in Dollars"]].to_numpy())
display(bf.records)

Exception: Attempted conversion to numpy array (dtype=float) failed. Reason could not convert string to float: '0.12/oz'

In [26]:
cost = m.addVariable('cost','free',description="total cost")
x = m.addVariable('x','integer',domain = Item,description="number of servings for each item")


In [27]:
%%gams
scalar
    requirements /3000/;

alias(Item, i);
    
equations
    obj "minimize total meal cost"
    cal_constraint_up
    cal_constraint_low
    protein_constraint_up
    protein_constraint_low
    carb_constraint_up
    carb_constraint_low
    fat_constraint_up
    fat_constraint_low
    calcium_constraint_up
    calcium_constraint_low
    cost_constraint
    meat_constraint_up
    meat_constraint_low
    fruit_constraint_up
    fruit_constraint_low
    veg_constraint_up
    veg_constraint_low
    breakfast_constraint_up
    breakfast_constraint_low
    frozen_constraint_up
    frozen_constraint_low
    drink_constraint_up
    drink_constraint_low
    dessert_constraint_up
    dessert_constraint_low
    dry_constraint_up
    dry_constraint_low
    dairy_constraint_up
    dairy_constraint_low;
    
obj..
    cost =e= sum(Item, price(Item)*x(Item));

# Do not go over budget
cost_constraint..
    cost =l= 100;
    
# Make sure we meet our nutritional goals
cal_constraint_up..
    sum(Item, x(Item) * calories(Item)) =l= 3500;
    
cal_constraint_low..
    sum(Item, x(Item) * calories(Item)) =g= 3000;
    
protein_constraint_up..
    sum(i, x(i) * protein(i)) =l= 73;
    
protein_constraint_low..
    sum(i, x(i) * protein(i)) =g= 45;
    
carb_constraint_up..
    sum(i, x(i) * carbs(i)) =l= 503;
    
carb_constraint_low..
    sum(i, x(i) * carbs(i)) =g= 308;
    
fat_constraint_up..
    sum(i, x(i) * fat(i)) =l= 120;
    
fat_constraint_low..
    sum(i, x(i) * fat(i)) =g= 69;
    
calcium_constraint_up..
    sum(i, x(i) * calcium(i)) =l= 200;
    
calcium_constraint_low..
    sum(i, x(i) * calcium(i)) =g= 100;
    
    
# Make sure we diversify the food groups we are eating     
meat_constraint_up..
    sum(meat, calories(meat) * x(meat)) =l= 1000;
    
meat_constraint_low..
    sum(meat, calories(meat) * x(meat)) =g= 100;
    
fruit_constraint_up..
    sum(fruit, calories(fruit) * x(fruit)) =l= 1000;
    
fruit_constraint_low..
    sum(fruit, calories(fruit) * x(fruit)) =g= 100;
    
veg_constraint_up..
    sum(vegetable, calories(vegetable) * x(vegetable)) =l= 1000;
    
veg_constraint_low..
    sum(vegetable, calories(vegetable) * x(vegetable)) =g= 100;
    
breakfast_constraint_up..
    sum(breakfast, calories(breakfast) * x(breakfast)) =l= 500;
    
breakfast_constraint_low..
    sum(breakfast, calories(breakfast) * x(breakfast)) =g= 50;
    
frozen_constraint_up..
    sum(frozen, calories(frozen) * x(frozen)) =l= 100;
    
frozen_constraint_low..
    sum(frozen, calories(frozen) * x(frozen)) =g= 0;
    
drink_constraint_up..
    sum(drink, calories(drink) * x(drink)) =l= 100;
    
drink_constraint_low..
    sum(drink, calories(drink) * x(drink)) =g= 0;
    
dessert_constraint_up..
    sum(dessert, calories(dessert) * x(dessert)) =l= 50;
    
dessert_constraint_low..
    sum(dessert, calories(dessert) * x(dessert)) =g= 0;
    
dry_constraint_up..
    sum(dry, calories(dry) * x(dry)) =l= 250;
    
dry_constraint_low..
    sum(dry, calories(dry) * x(dry)) =g= 20;
    
dairy_constraint_up..
    sum(dairy, calories(dairy) * x(dairy)) =l= 150;
    
dairy_constraint_low..
    sum(dairy, calories(dairy) * x(dairy)) =g= 0;
    
# X cannot be negative
x.lo(i) = 0;
 
model plan /obj 
    cal_constraint_up
    cal_constraint_low
    protein_constraint_up
    protein_constraint_low
    carb_constraint_up
    carb_constraint_low
    fat_constraint_up
    fat_constraint_low
    calcium_constraint_up
    calcium_constraint_low
    cost_constraint
    meat_constraint_up
    meat_constraint_low
    fruit_constraint_up
    fruit_constraint_low
    veg_constraint_up
    veg_constraint_low
    breakfast_constraint_up
    breakfast_constraint_low
    frozen_constraint_up
    frozen_constraint_low
    drink_constraint_up
    drink_constraint_low
    dessert_constraint_up
    dessert_constraint_low
    dry_constraint_up
    dry_constraint_low
    dairy_constraint_up
    dairy_constraint_low
    /;
solve plan using mip minimizing cost;



,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1.31,30,88,MIP,CPLEX,0.005


In [28]:
display(x.records)

,Item,level,marginal,lower,upper,scale
0,chicken,1.0,0.12,0.0,inf,1.0
1,yogurt,0.0,0.07,0.0,inf,1.0
2,cereal,0.0,0.27,0.0,inf,1.0
3,Corn,0.0,0.17,0.0,inf,1.0
4,Potatoes,1.0,0.05,0.0,inf,1.0
...,...,...,...,...,...,...
82,Ice cream,0.0,0.17,0.0,inf,1.0
83,cookies,0.0,0.20,0.0,inf,1.0
84,brownies,0.0,0.34,0.0,inf,1.0
85,Pizza rolls,0.0,0.19,0.0,inf,1.0


In [29]:
# Apply a 2.4% inflation rate to estimate 2024 prices
inflation_rate = 0.024
food_prices_df['Price 2024'] = food_prices_df['Price per Ounce'] * (1 + inflation_rate)

In [30]:
# Generate a distribution around the adjusted price for variability
np.random.seed(42)  # For reproducibility
price_variation_factor = 0.05  # 5% variation for the distribution

# Function to generate price distribution
def generate_price_distribution(price):
    return np.random.normal(price, price * price_variation_factor, 1000)

# Apply the function to generate distributions for 2024 prices
food_prices_df['Price Distribution 2024'] = food_prices_df['Price 2024'].apply(generate_price_distribution)

In [31]:
display(x.records[x.records["level"] !=0])

,Item,level,marginal,lower,upper,scale
0,chicken,1.0,0.12,0.0,inf,1.0
4,Potatoes,1.0,0.05,0.0,inf,1.0
9,Onions,1.0,0.07,0.0,inf,1.0
33,Pretzels,4.0,0.14,0.0,inf,1.0
38,Applesauce,1.0,0.07,0.0,inf,1.0
55,Pancake mix,1.0,0.07,0.0,inf,1.0
58,Peaches,1.0,0.09,0.0,inf,1.0
70,tea,2.0,0.02,0.0,inf,1.0
80,pita bread,1.0,0.24,0.0,inf,1.0
